# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import csv
import json
import os

import joblib
import requests
from azureml.core import (Environment, Experiment, RunConfiguration,
                          ScriptRunConfig, Workspace)
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig, Model

from azureml.core.webservice import AciWebservice, Webservice
from azureml.train.hyperdrive import (BanditPolicy, HyperDriveConfig,
                                      RandomParameterSampling, choice, normal,
                                      uniform)
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-udacity-stroke-experiment2'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
# Create compute cluster
cluster_name = "udacity-capstone-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_F8s_v2', # 'Standard_DS12_v2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.01, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"C": uniform(0.1, 1.0), 
                                 "max_iter": choice(50,100,150,200,250)})


#TODO: Create your estimator and hyperdrive config
# environment = Environment.get(workspace=ws, name=env_name)
environment = Environment.from_conda_specification(name='hyperparameter-env', file_path='hyperparameter_env.yml')

script_run_config = ScriptRunConfig(
                      source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=environment
                      )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(
                             run_config=script_run_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=15,
                             max_concurrent_runs=5)
                             

No Python version provided, defaulting to "3.8.13"


In [11]:
#TODO: Submit your experiment
experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810
Web View: https://ml.azure.com/runs/HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810?wsid=/subscriptions/ac113154-2e6f-499c-9be7-09bd5fba4d3a/resourcegroups/udacity-capstone-rg/workspaces/udacity-capstone&tid=8f47ad71-44ca-48bf-afe3-56b9360a4495

Streaming azureml-logs/hyperdrive.txt

[2022-09-23T09:46:50.923657][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2022-09-23T09:46:52.2352272Z][SCHEDULER][INFO]Scheduling job, id='HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810_1' 
[2022-09-23T09:46:52.3979031Z][SCHEDULER][INFO]Scheduling job, id='HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810_2' 
[2022-09-23T09:46:52.506848][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.
[2022-09-23T09:46:52.5858505Z][SCHEDULER][INFO]Scheduling job, id='HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810_3' 
[2022-09-23T09:46:52.5647044Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_14091134-0c4e-472e-b6ef-5d3ee

{'runId': 'HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810',
 'target': 'udacity-capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-23T09:46:49.855333Z',
 'endTimeUtc': '2022-09-23T09:54:56.84883Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '11f27fb5-4da2-4257-af15-654f8ae22aba',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9556425309849967',
  'best_child_run_id': 'HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810_11',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810_11'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_hyperdrive_run)

best_run_metrics = best_hyperdrive_run.get_metrics()
print(best_run_metrics)

print('RunID: ', best_hyperdrive_run.id)

Run(Experiment: hyperdrive-udacity-stroke-experiment2,
Id: HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810_11,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 0.8506754618748122, 'Max iterations:': 150, 'Accuracy': 0.9556425309849967}
RunID:  HD_14091134-0c4e-472e-b6ef-5d3ee4a1a810_11


In [15]:
best_model = Model.register(workspace=ws, 
                            model_path ="./outputs/model.pkl",
                            model_name = "best-hyperdrive-model",
                            tags = {"version": "1"},
                            description = "Udacity Stroke classification",
                          )

Registering model best-hyperdrive-model


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [3]:
env = Environment.from_conda_specification(name='hyperparameter-env', file_path='hyperparameter_env.yml')

python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']

for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

# for pip_package in ["scikit-learn"]:
#     env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script="score.py", environment = env)

No Python version provided, defaulting to "3.8.13"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [4]:
from azureml.core.webservice import LocalWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1.8, memory_gb = 4,enable_app_insights = True)

# deployment_config = LocalWebservice.deploy_configuration(port=6789)


print('Deploying model...')
service = Model.deploy(ws, "best-hyperdrive-model", [best_model], inference_config, deployment_config, overwrite=True, show_output = True)
service.wait_for_deployment(True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-30 11:24:32+00:00 Creating Container Registry if not exists.
2021-03-30 11:24:32+00:00 Registering the environment.
2021-03-30 11:24:34+00:00 Use the existing image.
2021-03-30 11:24:34+00:00 Generating deployment configuration.
2021-03-30 11:24:37+00:00 Submitting deployment to compute..
2021-03-30 11:24:43+00:00 Checking the status of deployment best-hyperdrive-model..
2021-03-30 11:29:54+00:00 Checking the status of inference endpoint best-hyperdrive-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
# Test Remote service
data = {
    "data":
    [
        {
            'id': "0",
            'bmi': "0",
            'age': "80",
            'gender': "1",
            'work_type': "0",
            'hypertension': "1",
            'heart_disease': "0",
            'Residence_type': "0",
            'smoking_status': "0",
            'ever_married': False,
            'avg_glucose_level': "0",
        },
      ]
    }
# Convert to JSON string
data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(data)

headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(service.scoring_uri, data, headers=headers)
print(resp.json())

{"result": [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

# Delete all resources not used
service.delete()
compute_target.delete()

.2021-03-38T86:55:26,229713888.00.88 - iot-server/run ,n2821-03-30T06:55:26,234636900.88:00 - rsyslog/run ...821-03-30T06:55:26,252211600+08:00 - nginx/run \\n/usr/sbin/nginx: /azureml-envs/azureml_220913aa74c252741cc997dchebOlc69/lib/libcrypto.so.1.8.0: no version information available (required by /usr/sbin/nginxl\\n/usr/sbin/nginx: /azureml-envs/a zureml 220613aa74c252741cc067dcbebOlc60/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx,,o/usr/sbin/nginx: /azureml-envs/azureml_220013aa 74c252741cc997dchebOlc69/lib/libssl.so.1.8.0: no version information available (required by /usr/sbin/n inxI\\n/usr/sbin/n inx. /az reml-envs/azureml 220613aa74c belc68/lib/libssl.so.1.8.0: no version information available (required by /usr/sbin/nginxI\\n/usr/sbin/ngginx: /azureml-envsg/a Uremlu238813aa74c2S3741cc887dcbeb01c2:82/74llic/a1= so.1.8.0: no version information available (required by /usr/sbin/nginx1782841-83-30T86:33:26,432183809860:013 - gunicorn/runz1nr

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

